<!--Header-->
<div>
    <div class="row" style="color: #4D4D4D;font-size: 15px;padding-bottom: 20px">
        <div class="col-md-7">
            <img src="http://materials.cv.uoc.edu/cdocent/common/img/logo-uoc.png" alt="Logo UOC" class="img-responsive" style="margin:20px 0px 0px">
        </div>
        <div class="col-md-5">
            <h1 style="margin:15px 0px 0px;font-size: 40px;"></h1>
            <h2>Global Observatory on Donation and Transplantation (Part II)</h2>
            <div style="text-align:left;margin-top: 5px;"></div>
        </div>
    </div>
    <div class="row" style="background: #FCB517;padding: 10px 20px;">
        <div class="col-md-6">
            <div style="text-align:right;">Autor: Jordi Puig Ovejero</div>
        </div>
    </div>
</div>
<!--/Header-->

## Índex ##
- [1. Creació de les noves dades](#importaci%C3%B3,-neteja-i-creaci%C3%B3-de-les-dades)
- [2. Procés de creació seguit](#el-proc%C3%A9s-de-creaci%C3%B3-seguit)
- [3. Característiquesdel joc de dades](#descripci%C3%B3-de-les-dades)
- [4, Les preguntes que respon la visualització](#an%C3%A0lisi-de-les-dades)
- [5. Reflexió: Aprenentatge i limitacions](#conclusions)
-  [6. Diccionari de les variables](#diccionari)


## 1. Creació de les noves dades ##

#### Lectura del fitxer de dades ####

### Transplant Rate per 100k population al 2021 ###
Total de trasplantaments per cada 100.000 habitants.

In [112]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [113]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Verificar los nombres de las columnas
print(df.columns)

# Filtrar el DataFrame para incluir solo el año 2021
df_2021 = df[df['REPORTYEAR'] == 2021]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Calcular el número total de trasplantes por fila
df_2021['Total Transplants'] = df_2021[required_columns].sum(axis=1)

# Convertir la población a formato numérico (eliminando comas si existen)
df_2021['POPULATION'] = df_2021['POPULATION'].str.replace(',', '').astype(float)

# Calcular la media de la población
average_population = df_2021['POPULATION'].mean()

# Calcular el número total de trasplantes
total_transplants = df_2021['Total Transplants'].sum()

# Calcular el ratio de trasplantes por persona y redondear a 2 decimales
transplants_per_capita = round(total_transplants / average_population, 2)

# Crear un DataFrame para almacenar el resultado
result_df = pd.DataFrame({
    'Total Transplants': [round(total_transplants, 2)],
    'Average Population': [round(average_population, 2)],
    'Transplants per Capita': [transplants_per_capita]
})

# Definir la ruta de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
result_df.to_csv(path + 'transplants_per_capita_overall_2021.csv', index=False)

# Mostrar los resultados
print(result_df)


Index(['REGION', 'COUNTRY', 'REPORTYEAR', 'POPULATION', 'TOTAL Actual DD',
       'Actual DBD', 'Actual DCD', 'Total Utilized DD', 'Utilized DBD',
       'Utilized DCD', 'DD Kidney Tx', 'LD Kidney Tx', 'TOTAL Kidney Tx',
       'DD Liver Tx', 'DOMINO Liver Tx', 'LD Liver Tx', 'TOTAL Liver TX',
       'Total Heart TX', 'DD Lung Tx', 'LD Lung Tx', 'TOTAL Lung Tx',
       'Pancreas Tx', 'Kidney Pancreas Tx', 'Small Bowel Tx'],
      dtype='object')
   Total Transplants  Average Population  Transplants per Capita
0           142107.0              308.45                  460.71


### Transplant Rate per 100k x country al 2021
Taxa de trasplantaments d'òrgans per cada 100.000 habitants a cada país o regió.

In [114]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Verificar los nombres de las columnas
print(df.columns)

# Filtrar el DataFrame para incluir solo el año 2021
df_2021 = df[df['REPORTYEAR'] == 2021]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Calcular el número total de trasplantes por fila
df_2021['Total Transplants'] = df_2021[required_columns].sum(axis=1)

# Convertir la población a formato numérico (eliminando comas si existen)
df_2021['POPULATION'] = df_2021['POPULATION'].str.replace(',', '').astype(float)

# Calcular la media de la población por país
average_population_by_country = df_2021.groupby('COUNTRY')['POPULATION'].mean().reset_index()

# Calcular el número total de trasplantes por país
total_transplants_by_country = df_2021.groupby('COUNTRY')['Total Transplants'].sum().reset_index()

# Combinar los dataframes de total de trasplantes y población media
combined_df = pd.merge(total_transplants_by_country, average_population_by_country, on='COUNTRY')

# Calcular el ratio de trasplantes por persona
combined_df['Transplants per Capita'] = combined_df['Total Transplants'] / combined_df['POPULATION']

# Path de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
combined_df.to_csv(path + 'transplants_per_capita_by_country.csv', index=False)

# Filtrar solo COUNTRY y Transplants per Capita para generar el archivo CSV y redondear a 2 decimales
combined_df = combined_df.round({'Transplants per Capita': 2})
combined_df = combined_df[['COUNTRY', 'Transplants per Capita']]
combined_df.to_csv(path + 'transplants_per_capita_by_country_filtered.csv', index=False)

# Mostrar los resultados
print(combined_df)


Index(['REGION', 'COUNTRY', 'REPORTYEAR', 'POPULATION', 'TOTAL Actual DD',
       'Actual DBD', 'Actual DCD', 'Total Utilized DD', 'Utilized DBD',
       'Utilized DCD', 'DD Kidney Tx', 'LD Kidney Tx', 'TOTAL Kidney Tx',
       'DD Liver Tx', 'DOMINO Liver Tx', 'LD Liver Tx', 'TOTAL Liver TX',
       'Total Heart TX', 'DD Lung Tx', 'LD Lung Tx', 'TOTAL Lung Tx',
       'Pancreas Tx', 'Kidney Pancreas Tx', 'Small Bowel Tx'],
      dtype='object')
                     COUNTRY  Transplants per Capita
0                    Albania                    0.72
1                  Argentina                    4.01
2                    Armenia                    6.00
3                  Australia                    5.41
4                    Austria                   71.89
..                       ...                     ...
86                  Türkiye                    58.21
87      United Arab Emirates                   17.90
88            United Kingdom                    5.88
89  United States of

### Donor Ratio by Transplant Type, Region and 2021
Proporció d'utilització de donants (sigui DBD o DCD) per a cada tipus de trasplantament d'òrgan (ronyó, fetge, cor, etc.).

In [115]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el año 2021
df_2021 = df[df['REPORTYEAR'] == 2021]

# Convertir la población a formato numérico (eliminando comas si existen)
df_2021['POPULATION'] = df_2021['POPULATION'].str.replace(',', '').astype(float)

# Reemplazar NaN con 0 para evitar errores en los cálculos
df_2021.fillna(0, inplace=True)

# Agrupar por región y sumar las columnas relevantes
grouped_df_2021 = df_2021.groupby('REGION').sum().reset_index()

# Calcular las proporciones de utilización de donantes para cada tipo de trasplante, manejando valores nulos y ceros
grouped_df_2021['Kidney Donor Utilization Ratio'] = (grouped_df_2021['DD Kidney Tx'] + grouped_df_2021['LD Kidney Tx']) / grouped_df_2021['TOTAL Kidney Tx']
grouped_df_2021['Liver Donor Utilization Ratio'] = (grouped_df_2021['DD Liver Tx'] + grouped_df_2021['LD Liver Tx']) / grouped_df_2021['TOTAL Liver TX']
grouped_df_2021['Heart Donor Utilization Ratio'] = grouped_df_2021['Total Heart TX'] / grouped_df_2021['Total Heart TX']
grouped_df_2021['Lung Donor Utilization Ratio'] = (grouped_df_2021['DD Lung Tx'] + grouped_df_2021['LD Lung Tx']) / grouped_df_2021['TOTAL Lung Tx']

# Reemplazar proporciones infinitas o indefinidas con 0
grouped_df_2021.replace([float('inf'), float('-inf'), pd.NA, pd.NaT], 0, inplace=True)

# Crear un DataFrame con las proporciones calculadas y redondear a 2 decimales
utilization_ratios_2021 = grouped_df_2021[['REGION', 'Kidney Donor Utilization Ratio', 'Liver Donor Utilization Ratio', 'Heart Donor Utilization Ratio', 'Lung Donor Utilization Ratio']].round(2)

# Path de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
utilization_ratios_2021.to_csv(path + 'donor_utilization_ratios_by_region_2021.csv', index=False)

# Mostrar los resultados
print(utilization_ratios_2021)


                  REGION  Kidney Donor Utilization Ratio  \
0                 Africa                             1.0   
1                America                             1.0   
2  Eastern Mediterranean                             1.0   
3                 Europe                             1.0   
4        South-East Asia                             1.0   
5        Western Pacific                             1.0   

   Liver Donor Utilization Ratio  Heart Donor Utilization Ratio  \
0                            NaN                            NaN   
1                            1.0                            1.0   
2                            1.0                            1.0   
3                            1.0                            1.0   
4                            1.0                            1.0   
5                            1.0                            1.0   

   Lung Donor Utilization Ratio  
0                           NaN  
1                           1.0  
2      

### Total Utilized LD al 2021
Total de donants difunts utilitzats, sumant LD Kidney Tx, LD Liver Tx i LD Lung Tx

In [116]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el año 2021
df_2021 = df[df['REPORTYEAR'] == 2021]

# Convertir la población a formato numérico (eliminando comas si existen)
df_2021['POPULATION'] = df_2021['POPULATION'].str.replace(',', '').astype(float)

# Reemplazar NaN con 0 para evitar errores en los cálculos
df_2021.fillna(0, inplace=True)

# Calcular el "Total Utilized LD"
df_2021['Total Utilized LD'] = df_2021['LD Kidney Tx'] + df_2021['LD Liver Tx'] + df_2021['LD Lung Tx']

# Agrupar por región y sumar las columnas relevantes
grouped_df_2021 = df_2021.groupby('REGION').sum().reset_index()

# Calcular las proporciones de utilización de donantes para cada tipo de trasplante, manejando valores nulos y ceros
grouped_df_2021['Kidney Donor Utilization Ratio'] = (grouped_df_2021['DD Kidney Tx'] + grouped_df_2021['LD Kidney Tx']) / grouped_df_2021['TOTAL Kidney Tx']
grouped_df_2021['Liver Donor Utilization Ratio'] = (grouped_df_2021['DD Liver Tx'] + grouped_df_2021['LD Liver Tx']) / grouped_df_2021['TOTAL Liver TX']
grouped_df_2021['Heart Donor Utilization Ratio'] = grouped_df_2021['Total Heart TX'] / grouped_df_2021['Total Heart TX']  # Este cálculo siempre da 1 porque dividimos por sí mismo
grouped_df_2021['Lung Donor Utilization Ratio'] = (grouped_df_2021['DD Lung Tx'] + grouped_df_2021['LD Lung Tx']) / grouped_df_2021['TOTAL Lung Tx']

# Reemplazar proporciones infinitas o indefinidas con 0
grouped_df_2021.replace([float('inf'), float('-inf'), pd.NA, pd.NaT], 0, inplace=True)

# Crear un DataFrame con las proporciones calculadas y redondear a 2 decimales
utilization_ratios_2021 = grouped_df_2021[['REGION', 'Total Utilized LD', 'Kidney Donor Utilization Ratio', 'Liver Donor Utilization Ratio', 'Heart Donor Utilization Ratio', 'Lung Donor Utilization Ratio']].round(2)

# Path de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
utilization_ratios_2021.to_csv(path + 'donor_utilization_LD_ratios_by_region_2021.csv', index=False)

# Mostrar los resultados
print(utilization_ratios_2021)


                  REGION  Total Utilized LD  Kidney Donor Utilization Ratio  \
0                 Africa              359.0                             1.0   
1                America             9774.0                             1.0   
2  Eastern Mediterranean             4118.0                             1.0   
3                 Europe             9048.0                             1.0   
4        South-East Asia            11154.0                             1.0   
5        Western Pacific             8663.0                             1.0   

   Liver Donor Utilization Ratio  Heart Donor Utilization Ratio  \
0                            NaN                            NaN   
1                            1.0                            1.0   
2                            1.0                            1.0   
3                            1.0                            1.0   
4                            1.0                            1.0   
5                            1.0            

## 2. Procés de creació seguit  #

### 1. Preparació de Dades

**Carregar i Netejar les Dades**

* Decisió de Disseny: Assegurar que les dades estiguin completes i correctament formatejades per evitar errors en l'anàlisi.
* Convertir columnes numèriques amb comes a format numèric.
* Omplir valors nuls amb zeros o gestionar-los adequadament.


**Calcular Mètriques Rellevants**

* Decisió de Disseny: Calcular mètriques que seran útils per a les visualitzacions i l'anàlisi posterior.
* Calcular "Total Utilized LD".
* Crear columna Total Transplants


### 2. Distribució Geogràfica dels Trasplantaments

**Agrupar per Regió i País**

* Decisió de Disseny: Utilitzar mapes o gràfics de barres per visualitzar les disparitats geogràfiques.
* Agrupar les dades per regió i país per obtenir el total de trasplantaments.

### 3. Tendències Temporals

**Anàlisi per Any**

* Decisió de Disseny: Utilitzar gràfics de línies per mostrar les tendències al llarg del temps.
* Agrupar per any i calcular el total de trasplantaments.

### 4. Proporció de Tipus de Donants i Trasplantaments

**Proporció de Donants per Tipus**

* Decisió de Disseny: Utilitzar gràfics de torta o barres per visualitzar les proporcions.
* Calcular les proporcions de trasplantaments realitzats amb diferents tipus de donants.

### 5. Relació entre la Població i el Nombre de Trasplantaments

**Anàlisi de la Relació Població-Trasplantaments**

* Decisió de Disseny: Utilitzar gràfics de dispersió per mostrar la relació.
* Calcular la relació entre la població i el nombre total de trasplantaments.

### 6. Comparació de Diferents Tipus de Trasplantaments

**Comparar Tipus de Trasplantaments per Regió/País**

* Decisió de Disseny: Utilitzar gràfics de barres apilades o comparatius.
* Agrupar i comparar els diferents tipus de trasplantaments per regió i país.

### 7. Eficiència en la Utilització de Donants

**Anàlisi d'Eficiència**

* Decisió de Disseny: Utilitzar gràfics de barres per comparar l'eficiència.
* Calcular l'eficiència en la utilització de donants.

### 8. Visualització i Creació de l'Infograma

**Selecció de Gràfics**

* Decisió de Disseny: Escollir el tipus de gràfic adequat per a cada anàlisi:
    - Mapes de calor per a la distribució geogràfica.
    - Gràfics de línies per a les tendències temporals.
     - Gràfics de torta o barres per a les proporcions.
     - Gràfics de dispersió per a les relacions.
     - Gràfics de barres apilades per a la comparació de tipus de trasplantaments.
     - Gràfics de barres per a l'eficiència en la utilització de donants.

## 3. Característiquesdel joc de dades

El joc de dades proporciona informació detallada sobre l'activitat de trasplantaments d'òrgans a nivell mundial, segmentada per regions i països. Aquí es presenta una breu descripció de les característiques més destacades:

0. **DIMENSIONS DEL JOC DE DADES:**
   - **Nombre de Files:** 3639
   - **Nombre de Columnes:** 24

1. **VARIABLES CATEGÒRIQUES:**
   - **REGION**: Aquesta variable categoritza els registres en funció de la regió geogràfica, per exemple, Africa, Europe, Eastern Mediterranean, entre d'altres.
   - **COUNTRY**: Identifica el país específic on s'ha realitzat el registre.
   - **REPORTYEAR**: Tot i ser una dada numèrica (any), es pot considerar com a categòrica per identificar el període temporal de cada registre.

2. **VARIABLES NUMÈRIQUES:**
   - **POPULATION**: Població del país en milions d'habitants.
   - **TOTAL Actual DD**: Total de donants difunts.
   - **Actual DBD**: Total de donants difunts per mort cerebral.
   - **Actual DCD**: Total de donants difunts per at

## 4. Les preguntes que respon la visualització  ##

Les preguntes que volem respondre en la o les visualitzacions:

1. **Distribució geogràfica dels trasplantaments**: que mostri la quantitat de trasplantaments d'òrgans diferents en diferents països o regions. Això ens ajudarà a identificar disparitats en l'accés als trasplantaments.

2. **Tendències temporals**: com ha variat el nombre de trasplantaments en el temps. Això ens permetrà identificar tendències al llarg dels anys.

3. **Proporció de tipus de donants i trasplantaments**: trobar la proporció de trasplantaments realitzats amb donants difunts per mort cerebral versus donants difunts per atur cardíac, així com la proporció de trasplantaments de diferents òrgans.

4. **Relació entre la població i el nombre de trasplantaments**: relació entre la població d'un país o regió i el nombre de trasplants duts a terme en aquesta àrea.

5. **Comparació de diferents tipus de trasplantaments**: comparació entre la quantitat de trasplantaments de ronyó, fetge, cor, pduts a terme, realitzats en diferents països o regions.

6. **Eficiència en la utilització de donants**: comparació entre els totals de donants disponibles amb els trduts a terments realitzats per avaluar l'eficiència en la utilització dels òrgans donats en iferents països o regions.

### 4.1. Els Trasplants al món

#### Total de trasplants per Regió al 2021

In [117]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el año 2021 y la región 'Europe'
df_2021_europe = df[(df['REPORTYEAR'] == 2021) & (df['REGION'] == 'Europe')]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Calcular el número total de trasplantes por país sumando las columnas requeridas
df_2021_europe['TOTAL Transplants'] = df_2021_europe[required_columns].sum(axis=1)

# Calcular el número total de trasplantes por país
total_transplants_by_country = df_2021_europe.groupby('COUNTRY')['TOTAL Transplants'].sum().reset_index()

# Filtrar los países que tienen cero trasplantes
total_transplants_by_country = total_transplants_by_country[total_transplants_by_country['TOTAL Transplants'] != 0]

# Guardar el resultado en un archivo CSV en la carpeta ./data/out/europe/
output_folder = './data/out/world/'
total_transplants_by_country.to_csv(output_folder + 'total_transplants_by_country_filtered_2021.csv', index=False)

# Mostrar los resultados
print(total_transplants_by_country)


                   COUNTRY  TOTAL Transplants
0                  Albania               21.0
1                  Armenia               18.0
2                  Austria              647.0
3                  Belarus              481.0
4                  Belgium              831.0
5   Bosnia and Herzegovina               11.0
6                 Bulgaria               48.0
7                  Croatia              271.0
8                   Cyprus               19.0
9           Czech Republic              768.0
10                 Denmark              348.0
11                 Estonia               54.0
12                 Finland              389.0
13                  France             5214.0
14                 Georgia               37.0
15                 Germany             3438.0
16                  Greece              217.0
17                 Hungary              301.0
18                 Iceland                9.0
19                 Ireland              204.0
20                  Israel        

#### Total de trasplants per Tipologia (per càpita)

In [118]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el año 2021 y la región 'Europe'
df_2021_europe = df[(df['REPORTYEAR'] == 2021) & (df['REGION'] == 'Europe')]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Calcular el número total de trasplantes por país y por tipología
total_transplants_by_country = df_2021_europe.groupby('COUNTRY')[required_columns].sum().reset_index()

# Filtrar los países que tienen cero trasplantes en todos los tipos de órganos
total_transplants_by_country = total_transplants_by_country[(total_transplants_by_country[required_columns] != 0).any(axis=1)]

# Renombrar las columnas para mayor claridad
total_transplants_by_country.rename(columns={
    'TOTAL Liver TX': 'Liver',
    'TOTAL Kidney Tx': 'Kidney',
    'TOTAL Lung Tx': 'Lung',
    'Total Heart TX': 'Heart'
}, inplace=True)

# Path de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
total_transplants_by_country.to_csv(path+ '/transplants_by_country_filtered_2021.csv', index=False)

# Mostrar los resultados
print(total_transplants_by_country)


                   COUNTRY   Liver  Kidney   Lung  Heart
0                  Albania     0.0    21.0    0.0    0.0
1                  Armenia     2.0    16.0    0.0    0.0
2                  Austria   159.0   306.0  123.0   59.0
3                  Belarus    91.0   336.0    6.0   48.0
4                  Belgium   268.0   417.0   94.0   52.0
5   Bosnia and Herzegovina     1.0    10.0    0.0    0.0
6                 Bulgaria    13.0    33.0    0.0    2.0
7                  Croatia   104.0   124.0    9.0   34.0
8                   Cyprus     0.0    19.0    0.0    0.0
9           Czech Republic   186.0   456.0   52.0   74.0
10                 Denmark    50.0   252.0   22.0   24.0
11                 Estonia     4.0    48.0    2.0    0.0
12                 Finland    75.0   268.0   24.0   22.0
13                  France  1225.0  3252.0  322.0  415.0
14                 Georgia    15.0    22.0    0.0    0.0
15                 Germany   834.0  1992.0  283.0  329.0
16                  Greece    2

#### Evoluació temporal dels donants (pmp)

In [119]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar columnas relevantes
df = df[['REGION', 'REPORTYEAR', 'TOTAL Actual DD', 'POPULATION']]

# Convertir la población a formato numérico (eliminar comas si existen)
df['POPULATION'] = df['POPULATION'].str.replace(',', '').astype(float)

# Calcular los donantes por millón de personas
df['Donors per Thousand'] = (df['TOTAL Actual DD'] / df['POPULATION']) * 1_000

# Redondear los valores a dos decimales
df['Donors per Thousand'] = df['Donors per Thousand'].round(0)  # Redondear a cero decimales

# Pivoteo para tener los años como columnas y las regiones como índices
pivot_df = df.pivot_table(index='REGION', columns='REPORTYEAR', values='Donors per Thousand', aggfunc='sum', fill_value=0)

# Ordenar las columnas por año
pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)

# Mostrar los resultados
print(pivot_df)

# Path de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
pivot_df.to_csv(path + 'total_donors_per_million_by_region_and_year.csv')


REPORTYEAR                2000     2001     2002     2003     2004     2005  \
REGION                                                                        
Africa                     0.0      0.0      0.0      0.0      0.0    156.0   
America                 3677.0  15675.0   3459.0  23322.0  32632.0  12897.0   
Eastern Mediterranean     30.0     60.0     72.0    177.0    461.0    973.0   
Europe                 57207.0  65415.0  84273.0  77905.0  84589.0  66598.0   
South-East Asia            0.0      0.0      0.0      0.0      0.0     72.0   
Western Pacific         1032.0    933.0   1056.0   1935.0   2121.0   9025.0   

REPORTYEAR                2006     2007      2008     2009  ...      2013  \
REGION                                                      ...             
Africa                   139.0    130.0     164.0    279.0  ...     133.0   
America                36237.0  11970.0   13457.0  18978.0  ...   13540.0   
Eastern Mediterranean   1126.0   1049.0    1279.0   2497.0 

#### Evoluació temporal dels tipus de donants (pMp)

In [120]:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Verificar las columnas de donantes actuales
donor_columns = ['TOTAL Actual DD', 'Actual DBD', 'Actual DCD']

# Crear un dataframe para almacenar los resultados
result_df = pd.DataFrame(columns=['Donor Type', 'REPORTYEAR', 'Total'])

# Calcular los totales de donantes por tipo y año
for donor_type in donor_columns:
    donor_totals_by_year = df.groupby('REPORTYEAR')[donor_type].sum().reset_index()
    donor_totals_by_year['Donor Type'] = donor_type
    donor_totals_by_year = donor_totals_by_year.rename(columns={donor_type: 'Total'})
    
    result_df = pd.concat([result_df, donor_totals_by_year], ignore_index=True)

# Pivotar el dataframe para tener años como columnas y tipos de donant como filas
pivot_df = result_df.pivot(index='Donor Type', columns='REPORTYEAR', values='Total')

# Rellenar valores nulos con 0
pivot_df = pivot_df.fillna(0)

# Mostrar los resultados
print(pivot_df)

# Path de salida
path = './data/out/world/'

# Guardar el resultado en un archivo CSV
pivot_df.to_csv(path + 'total_actual_donors_by_type_and_year.csv', index=True)

REPORTYEAR          2000     2001     2002     2003     2004     2005  \
Donor Type                                                              
Actual DBD        9703.0  10703.0   9753.0  14776.0  18710.0  19590.0   
Actual DCD         118.0    167.0    242.0    518.0    699.0   1493.0   
TOTAL Actual DD  11181.0  12207.0  11405.0  15294.0  19409.0  20681.0   

REPORTYEAR          2006     2007     2008     2009  ...     2013     2014  \
Donor Type                                           ...                     
Actual DBD       10072.0  18700.0  17379.0  17264.0  ...  23889.0  24645.0   
Actual DCD        1132.0   1486.0   1573.0   1781.0  ...   2488.0   2542.0   
TOTAL Actual DD  21175.0  21446.0  22350.0  22906.0  ...  26929.0  27804.0   

REPORTYEAR          2015     2016     2017     2018     2019     2020  \
Donor Type                                                              
Actual DBD       26670.0  28239.0  30388.0  30703.0  32453.0  27936.0   
Actual DCD        5328.0

#### Trasplants DCD vs Donants (pMp)

In [121]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el año 2021
df_2021 = df[df['REPORTYEAR'] == 2021]

# Agrupar por región y calcular los totales de DCD utilizados y DCD totales
totals_2021 = df_2021.groupby('REGION').agg({
    'Utilized DCD': 'sum',
    'Actual DCD': 'sum'
}).reset_index()

# Path de salida
path = './data/out/world/'

# Guardar los resultados en un archivo CSV
totals_2021.to_csv(path + 'totals_dcd_by_region_2021.csv', index=False)

# Mostrar los resultados por consola
print(totals_2021)


                  REGION  Utilized DCD  Actual DCD
0                 Africa           0.0         0.0
1                America        3527.0      4423.0
2  Eastern Mediterranean           0.0         0.0
3                 Europe        1811.0      1992.0
4        South-East Asia           0.0         0.0
5        Western Pacific        2105.0      2131.0


#### Trasplants DBD vs Donants (pMp)

In [122]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el año 2021
df_2021 = df[df['REPORTYEAR'] == 2021]

# Agrupar por región y calcular los totales de DBD utilizados y DBD totales
totals_2021 = df_2021.groupby('REGION').agg({
    'Utilized DBD': 'sum',
    'Actual DBD': 'sum'
}).reset_index()

 # Path de salida
path = './data/out/world/'

# Guardar los resultados en un archivo CSV
totals_2021.to_csv(path + 'totals_dbd_by_region_2021.csv', index=False)

# Mostrar los resultados por consola
print(totals_2021)


                  REGION  Utilized DBD  Actual DBD
0                 Africa           0.0         0.0
1                America       13602.0     14572.0
2  Eastern Mediterranean        1104.0      1158.0
3                 Europe        8384.0      8925.0
4        South-East Asia         759.0       776.0
5        Western Pacific        4153.0      4180.0


### 4.2. Els Trasplants a Europa

#### Total de trasplants per Tipologia i País al 2021

In [123]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el año 2021 y la región 'Europe'
df_2021_europe = df[(df['REPORTYEAR'] == 2021) & (df['REGION'] == 'Europe')]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Renombrar las columnas para que tengan nombres más amigables
df_2021_europe = df_2021_europe.rename(columns={
    'TOTAL Liver TX': 'Liver',
    'TOTAL Kidney Tx': 'Kidney',
    'TOTAL Lung Tx': 'Lung',
    'Total Heart TX': 'Heart'
})

# Seleccionar solo las columnas necesarias
df_2021_europe = df_2021_europe[['COUNTRY', 'Liver', 'Kidney', 'Lung', 'Heart']]

# Calcular el número total de trasplantes por país sumando las columnas requeridas
df_2021_europe['Total Transplants'] = df_2021_europe[['Liver', 'Kidney', 'Lung', 'Heart']].sum(axis=1)

# Filtrar los países que tienen cero trasplantes en todas las categorías
df_2021_europe = df_2021_europe[df_2021_europe['Total Transplants'] != 0]

# Eliminar la columna de total de trasplantes ya que no se necesita en el archivo de salida
df_2021_europe = df_2021_europe.drop(columns=['Total Transplants'])

# Crear una carpeta de salida si no existe
output_folder = './data/out/europe/'

# Guardar el DataFrame en un archivo CSV
output_file = output_folder + 'total_transplants_by_country_and_type_2021.csv'
df_2021_europe.to_csv(output_file, index=False)

# Mostrar los resultados
print(df_2021_europe)


                     COUNTRY   Liver  Kidney   Lung  Heart
3469                 Armenia     2.0    16.0    0.0    0.0
3470                 Belarus    91.0   336.0    6.0   48.0
3474                 Georgia    15.0    22.0    0.0    0.0
3475                 Iceland     0.0     9.0    0.0    0.0
3476                  Israel    96.0   476.0   55.0   24.0
3477     Republic of Moldova     4.0     0.0    NaN    0.0
3483                  Norway    98.0   231.0   24.0   23.0
3484                  Serbia     3.0     5.0    0.0    1.0
3485             Switzerland   151.0   362.0   42.0   33.0
3489                 Germany   834.0  1992.0  283.0  329.0
3490                 Austria   159.0   306.0  123.0   59.0
3491                 Belgium   268.0   417.0   94.0   52.0
3492                Bulgaria    13.0    33.0    0.0    2.0
3493                  Cyprus     NaN    19.0    NaN    NaN
3494                 Estonia     4.0    48.0    2.0    NaN
3495                 Finland    75.0   268.0   24.0   22

#### Evoluació temporal dels trasplants per tipus

In [124]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo la región 'Europe'
df_europe = df[df['REGION'] == 'Europe']

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Agrupar por año y sumar los trasplantes por tipo
transplants_by_year = df_europe.groupby('REPORTYEAR')[required_columns].sum().reset_index()

# Transponer los datos para que los tipos de trasplantes sean filas
transposed_df = transplants_by_year.set_index('REPORTYEAR').T.reset_index()
transposed_df.rename(columns={'index': 'Transplant Type'}, inplace=True)

# Guardar el resultado en un archivo CSV
output_folder = './data/out/europe/'
transposed_df.to_csv(output_folder + 'transplants_by_type_and_year_in_europe_transposed.csv', index=False)

# Mostrar los resultados
print(transposed_df)


REPORTYEAR  Transplant Type    2000    2001    2002     2003     2004  \
0            TOTAL Liver TX  4362.0  5354.0  5547.0   5845.0   6368.0   
1           TOTAL Kidney Tx  7754.0  8547.0  8241.0  16336.0  17831.0   
2             TOTAL Lung Tx   676.0   796.0  1106.0   1029.0   1100.0   
3            Total Heart TX  2019.0  2206.0  2064.0   2137.0   2126.0   

REPORTYEAR     2005     2006     2007     2008  ...     2013     2014  \
0            6618.0   6831.0   7300.0   7436.0  ...   9126.0   9349.0   
1           18267.0  18580.0  19864.0  19806.0  ...  24834.0  25383.0   
2            1210.0   1315.0   1345.0   1411.0  ...   1994.0   1998.0   
3               0.0   2227.0   2202.0   2170.0  ...   2393.0   2523.0   

REPORTYEAR     2015     2016     2017     2018     2019     2020     2021  \
0            9607.0   9990.0  10365.0  10556.0  10794.0   9257.0   9333.0   
1           26165.0  26726.0  27283.0  28080.0  28189.0  22006.0  23593.0   
2            2010.0   2090.0   2210.0

#### Evoluació temporal dels donants (pmp)

In [125]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar columnas relevantes
df = df[['REGION', 'REPORTYEAR', 'TOTAL Actual DD', 'POPULATION']]

# Filtrar el DataFrame para incluir solo la región 'Europe'
df = df[df['REGION'] == 'Europe']

# Convertir la población a formato numérico (eliminar comas si existen)
df['POPULATION'] = df['POPULATION'].str.replace(',', '').astype(float)

# Calcular los donantes por millón de personas
df['Donors per Million'] = (df['TOTAL Actual DD'] / df['POPULATION']) * 1_000

# Redondear los valores a cero decimales
df['Donors per Million'] = df['Donors per Million'].round(0)

# Pivoteo para tener los años como columnas y las regiones como índices
pivot_df = df.pivot_table(index='REGION', columns='REPORTYEAR', values='Donors per Million', aggfunc='sum', fill_value=0)

# Ordenar las columnas por año
pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)

# Transponer el DataFrame
transposed_df = pivot_df.transpose()

# Mostrar los resultados
print(transposed_df)

# Crear una carpeta de salida si no existe
output_folder = './data/out/europe/'
os.makedirs(output_folder, exist_ok=True)

# Guardar el resultado en un archivo CSV
output_file = output_folder + 'total_donors_per_million_by_region_and_year_transposed.csv'
transposed_df.to_csv(output_file)


REGION        Europe
REPORTYEAR          
2000         57207.0
2001         65415.0
2002         84273.0
2003         77905.0
2004         84589.0
2005         66598.0
2006         59508.0
2007         90001.0
2008        112374.0
2009         81268.0
2010         96638.0
2011         82780.0
2012        130704.0
2013        129810.0
2014         83820.0
2015         86162.0
2016         92659.0
2017        110730.0
2018         93473.0
2019        110760.0
2020         76169.0
2021         90518.0
2022         82808.0


#### Evoluació temporal dels tipus de donants (pMp)

In [126]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo la región 'Europe'
df = df[df['REGION'] == 'Europe']

# Verificar las columnas de donantes actuales
donor_columns = ['TOTAL Actual DD', 'Actual DBD', 'Actual DCD']

# Crear un dataframe para almacenar los resultados
result_df = pd.DataFrame(columns=['Donor Type', 'REPORTYEAR', 'Total'])

# Calcular los totales de donantes por tipo y año
for donor_type in donor_columns:
    donor_totals_by_year = df.groupby('REPORTYEAR')[donor_type].sum().reset_index()
    donor_totals_by_year['Donor Type'] = donor_type
    donor_totals_by_year = donor_totals_by_year.rename(columns={donor_type: 'Total'})
    
    result_df = pd.concat([result_df, donor_totals_by_year], ignore_index=True)

# Pivotar el dataframe para tener años como columnas y tipos de donant como filas
pivot_df = result_df.pivot(index='Donor Type', columns='REPORTYEAR', values='Total')

# Rellenar valores nulos con 0
pivot_df = pivot_df.fillna(0)

# Mostrar los resultados
print(pivot_df)

# Guardar el resultado en un archivo CSV
pivot_df.to_csv('./data/out/europe/total_actual_donors_by_type_and_year.csv', index=True)


REPORTYEAR         2000    2001    2002    2003    2004    2005    2006  \
Donor Type                                                                
Actual DBD       3161.0  4149.0  3085.0  7851.0  8356.0  8626.0  1714.0   
Actual DCD          0.0     0.0    52.0   247.0   302.0   837.0   367.0   
TOTAL Actual DD  4506.0  5484.0  4546.0  8098.0  8658.0  8973.0  9123.0   

REPORTYEAR         2007    2008    2009  ...     2013     2014     2015  \
Donor Type                               ...                              
Actual DBD       8898.0  8851.0  7040.0  ...  10092.0  10375.0  10447.0   
Actual DCD        547.0   543.0   661.0  ...   1150.0   1107.0   1209.0   
TOTAL Actual DD  9440.0  9566.0  9769.0  ...  11242.0  11492.0  12013.0   

REPORTYEAR          2016     2017     2018     2019     2020     2021     2022  
Donor Type                                                                      
Actual DBD       10950.0  11335.0  11379.0  11242.0   9053.0   8925.0   9166.0  
Actua

#### Trasplants DCD vs Donants (pMp)

In [127]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el año 2021 y la región 'Europe'
df_2021_europe = df[(df['REPORTYEAR'] == 2021) & (df['REGION'] == 'Europe')]

# Agrupar por país y calcular los totales de DCD utilizados y DCD totales
totals_2021_europe = df_2021_europe.groupby('COUNTRY').agg({
    'Utilized DCD': 'sum',
    'Actual DCD': 'sum'
}).reset_index()

# Filtrar los países donde ambos valores (Utilized DCD y Actual DCD) son mayores que cero
totals_2021_europe = totals_2021_europe[(totals_2021_europe['Utilized DCD'] > 0) | (totals_2021_europe['Actual DCD'] > 0)]

# Guardar los resultados en un archivo CSV
totals_2021_europe.to_csv('./data/out/europe/totals_dcd_by_country_in_europe_2021_filtered.csv', index=False)

# Mostrar los resultados por consola
print(totals_2021_europe)


           COUNTRY  Utilized DCD  Actual DCD
2          Austria           6.0         9.0
4          Belgium         121.0       135.0
9   Czech Republic          10.0        13.0
12         Finland           4.0         4.0
13          France         216.0       222.0
19         Ireland           8.0         8.0
20          Israel           3.0         3.0
21           Italy          80.0        93.0
24       Lithuania           4.0         4.0
27     Netherlands         172.0       174.0
30        Portugal          14.0        17.0
36           Spain         582.0       662.0
37          Sweden          27.0        27.0
38     Switzerland          53.0        57.0
40  United Kingdom         511.0       564.0


#### Trasplants DBD vs Donants (pMp)

In [128]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el año 2021 y la región 'Europe'
df_2021_europe = df[(df['REPORTYEAR'] == 2021) & (df['REGION'] == 'Europe')]

# Agrupar por país y calcular los totales de DBD utilizados y DCD totales
totals_2021_europe = df_2021_europe.groupby('COUNTRY').agg({
    'Utilized DBD': 'sum',
    'Actual DBD': 'sum'
}).reset_index()

# Filtrar los países donde ambos valores (Utilized DBD y Actual DBD) son mayores que cero
totals_2021_europe = totals_2021_europe[(totals_2021_europe['Utilized DBD'] > 0) | (totals_2021_europe['Actual DBD'] > 0)]

# Guardar los resultados en un archivo CSV
totals_2021_europe.to_csv('./data/out/europe/totals_dbd_by_country_in_europe_2021_filtered.csv', index=False)

# Mostrar los resultados por consola
print(totals_2021_europe)


                   COUNTRY  Utilized DBD  Actual DBD
2                  Austria         162.0       173.0
3                  Belarus         211.0       218.0
4                  Belgium         159.0       177.0
5   Bosnia and Herzegovina           1.0         1.0
6                 Bulgaria          14.0        17.0
7                  Croatia         112.0       121.0
8                   Cyprus           5.0         5.0
9           Czech Republic         232.0       255.0
10                 Denmark         101.0       105.0
11                 Estonia          21.0        21.0
12                 Finland         115.0       118.0
13                  France        1373.0      1392.0
15                 Germany         900.0       933.0
16                  Greece          45.0        52.0
17                 Hungary         101.0       102.0
18                 Iceland          11.0        11.0
19                 Ireland          54.0        57.0
20                  Israel          88.0      

#### Total de trasplants vs Donants per País al 2021

In [129]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el año 2021 y la región 'Europe'
df_2021_europe = df[(df['REPORTYEAR'] == 2021) & (df['REGION'] == 'Europe')]

# Agrupar por país y calcular los totales de DD utilizados y DD totales
totals_2021_europe = df_2021_europe.groupby('COUNTRY').agg({
    'Total Utilized DD': 'sum',
    'TOTAL Actual DD': 'sum'
}).reset_index()

# Filtrar los países donde ambos valores (Total Utilized DD y TOTAL Actual DD) son mayores que cero
totals_2021_europe = totals_2021_europe[(totals_2021_europe['Total Utilized DD'] > 0) | (totals_2021_europe['TOTAL Actual DD'] > 0)]

# Guardar los resultados en un archivo CSV
totals_2021_europe.to_csv('./data/out/europe/totals_dd_by_country_in_europe_2021_filtered.csv', index=False)

# Mostrar los resultados por consola
print(totals_2021_europe)


                   COUNTRY  Total Utilized DD  TOTAL Actual DD
2                  Austria              168.0            182.0
3                  Belarus              211.0            218.0
4                  Belgium              280.0            312.0
5   Bosnia and Herzegovina                1.0              1.0
6                 Bulgaria               14.0             17.0
7                  Croatia              112.0            121.0
8                   Cyprus                5.0              5.0
9           Czech Republic              242.0            268.0
10                 Denmark              101.0            105.0
11                 Estonia               21.0             21.0
12                 Finland              119.0            122.0
13                  France             1589.0           1614.0
15                 Germany              900.0            933.0
16                  Greece               45.0             52.0
17                 Hungary              101.0          

### 4.3. Els trasplants a Espanya

#### Total de trasplants per Tipologia al 2021

In [130]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el año 2021 y el país 'Spain'
df_2021_spain = df[(df['REPORTYEAR'] == 2021) & (df['COUNTRY'] == 'Spain')]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Renombrar las columnas para que tengan nombres más amigables
df_2021_spain = df_2021_spain.rename(columns={
    'TOTAL Liver TX': 'Liver',
    'TOTAL Kidney Tx': 'Kidney',
    'TOTAL Lung Tx': 'Lung',
    'Total Heart TX': 'Heart'
})

# Seleccionar solo las columnas necesarias
df_2021_spain = df_2021_spain[['COUNTRY', 'Liver', 'Kidney', 'Lung', 'Heart']]

# Calcular el número total de trasplantes por país sumando las columnas requeridas
df_2021_spain['Total Transplants'] = df_2021_spain[['Liver', 'Kidney', 'Lung', 'Heart']].sum(axis=1)

# Eliminar la columna de total de trasplantes ya que no se necesita en el archivo de salida
df_2021_spain = df_2021_spain.drop(columns=['Total Transplants'])

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar el DataFrame en un archivo CSV
output_file = output_folder + 'total_transplants_by_type_2021_spain.csv'
df_2021_spain.to_csv(output_file, index=False)

# Mostrar los resultados
print(df_2021_spain)


     COUNTRY   Liver  Kidney   Lung  Heart
3515   Spain  1078.0  2950.0  362.0  302.0


#### Evolució temporal dels trasplants per tipus

In [131]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo los años de 2000 a 2022 y el país 'Spain'
df_spain = df[(df['REPORTYEAR'] >= 2000) & (df['REPORTYEAR'] <= 2022) & (df['COUNTRY'] == 'Spain')]

# Definir las columnas requeridas para el cálculo
required_columns = ['TOTAL Liver TX', 'TOTAL Kidney Tx', 'TOTAL Lung Tx', 'Total Heart TX']

# Renombrar las columnas para que tengan nombres más amigables
df_spain = df_spain.rename(columns={
    'TOTAL Liver TX': 'Liver',
    'TOTAL Kidney Tx': 'Kidney',
    'TOTAL Lung Tx': 'Lung',
    'Total Heart TX': 'Heart'
})

# Seleccionar solo las columnas necesarias
df_spain = df_spain[['REPORTYEAR', 'COUNTRY', 'Liver', 'Kidney', 'Lung', 'Heart']]

# Calcular el número total de trasplantes por año y por país sumando las columnas requeridas
df_spain['Total Transplants'] = df_spain[['Liver', 'Kidney', 'Lung', 'Heart']].sum(axis=1)

# Eliminar la columna de total de trasplantes ya que no se necesita en el archivo de salida
df_spain = df_spain.drop(columns=['Total Transplants'])

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar el DataFrame en un archivo CSV
output_file = output_folder + 'total_transplants_by_type_spain_2000_2022.csv'
df_spain.to_csv(output_file, index=False)

# Mostrar los resultados
print(df_spain)


      REPORTYEAR COUNTRY   Liver  Kidney   Lung  Heart
2412        2000   Spain   954.0  1938.0  138.0  353.0
2413        2001   Spain   972.0  1924.0  143.0  341.0
2414        2002   Spain  1033.0  2032.0  161.0  310.0
2415        2003   Spain  1037.0  2051.0  145.0  290.0
2416        2004   Spain  1040.0  2126.0  143.0  294.0
2417        2005   Spain  1070.0  2200.0  167.0    NaN
2418        2006   Spain  1051.0  2157.0  169.0  274.0
2419        2007   Spain  1112.0  2211.0  185.0  241.0
2420        2008   Spain  1108.0  2229.0  192.0  292.0
2421        2009   Spain  1099.0  2328.0  219.0  274.0
2422        2010   Spain   971.0  2225.0  235.0  243.0
2423        2011   Spain  1137.0  2498.0  230.0  237.0
2424        2012   Spain  1084.0  2551.0  238.0  247.0
2425        2013   Spain  1093.0  2552.0  285.0  249.0
2426        2014   Spain  1068.0  2678.0  262.0  265.0
2935        2015   Spain  1162.0  2905.0  294.0  299.0
3076        2016   Spain  1159.0  2997.0  307.0  281.0
3165      

#### Evoluació temporal dels donants (pmp)

In [132]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo los años de 2000 a 2022 y el país 'Spain'
df_spain = df[(df['REPORTYEAR'] >= 2000) & (df['REPORTYEAR'] <= 2022) & (df['COUNTRY'] == 'Spain')]

# Filtrar columnas relevantes
df_spain = df_spain[['COUNTRY', 'REPORTYEAR', 'TOTAL Actual DD', 'POPULATION']]

# Convertir la población a formato numérico (eliminar comas si existen)
df_spain['POPULATION'] = df_spain['POPULATION'].str.replace(',', '').astype(float)

# Calcular los donantes por millón de personas
df_spain['Donors per Million'] = (df_spain['TOTAL Actual DD'] / df_spain['POPULATION']) * 1_000_000

# Redondear los valores a dos decimales
df_spain['Donors per Million'] = df_spain['Donors per Million'].round(0)  # Redondear a cero decimales

# Pivoteo para tener los años como columnas
pivot_df_spain = df_spain.pivot_table(index='COUNTRY', columns='REPORTYEAR', values='Donors per Million', aggfunc='sum', fill_value=0)

# Ordenar las columnas por año
pivot_df_spain = pivot_df_spain.reindex(sorted(pivot_df_spain.columns), axis=1)

# Mostrar los resultados
print(pivot_df_spain)

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar el resultado en un archivo CSV
output_file_spain = output_folder + 'total_donors_per_million_by_year_spain.csv'
pivot_df_spain.to_csv(output_file_spain)


REPORTYEAR       2000       2001       2002       2003       2004       2005  \
COUNTRY                                                                        
Spain       3396465.0  3345865.0  3531328.0  3510949.0  3637470.0  3587007.0   

REPORTYEAR       2006       2007       2008       2009  ...       2013  \
COUNTRY                                                 ...              
Spain       3476959.0  3555046.0  3535874.0  3576837.0  ...  3528785.0   

REPORTYEAR       2014       2015       2016       2017       2018       2019  \
COUNTRY                                                                        
Spain       3571125.0  4015184.0  4379610.0  4704741.0  4829741.0  4961207.0   

REPORTYEAR       2020       2021       2022  
COUNTRY                                      
Spain       3797009.0  4079229.0  4702355.0  

[1 rows x 23 columns]


#### Evoluació temporal dels tipus de donants (pMp)

In [133]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar el DataFrame para incluir solo el país 'Spain'
df = df[df['COUNTRY'] == 'Spain']

# Verificar las columnas de donantes actuales
donor_columns = ['TOTAL Actual DD', 'Actual DBD', 'Actual DCD']

# Crear un dataframe para almacenar los resultados
result_df = pd.DataFrame(columns=['Donor Type', 'REPORTYEAR', 'Total'])

# Calcular los totales de donantes por tipo y año
for donor_type in donor_columns:
    donor_totals_by_year = df.groupby('REPORTYEAR')[donor_type].sum().reset_index()
    donor_totals_by_year['Donor Type'] = donor_type
    donor_totals_by_year = donor_totals_by_year.rename(columns={donor_type: 'Total'})
    
    result_df = pd.concat([result_df, donor_totals_by_year], ignore_index=True)

# Pivotar el dataframe para tener años como columnas y tipos de donant como filas
pivot_df = result_df.pivot(index='Donor Type', columns='REPORTYEAR', values='Total')

# Rellenar valores nulos con 0
pivot_df = pivot_df.fillna(0)

# Mostrar los resultados
print(pivot_df)

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar el resultado en un archivo CSV
output_file = output_folder + 'total_actual_donors_by_type_and_year_spain.csv'
pivot_df.to_csv(output_file, index=True)


REPORTYEAR         2000    2001    2002    2003    2004    2005    2006  \
Donor Type                                                                
Actual DBD          0.0     0.0     0.0  1387.0  1424.0  1475.0     0.0   
Actual DCD          0.0     0.0     0.0    56.0    71.0    71.0    76.0   
TOTAL Actual DD  1345.0  1335.0  1409.0  1443.0  1495.0  1546.0  1509.0   

REPORTYEAR         2007    2008    2009  ...    2013    2014    2015    2016  \
Donor Type                               ...                                   
Actual DBD       1462.0  1500.0  1499.0  ...  1496.0  1489.0  1537.0  1524.0   
Actual DCD         88.0    77.0   107.0  ...   159.0   193.0   314.0   495.0   
TOTAL Actual DD  1550.0  1577.0  1606.0  ...  1655.0  1682.0  1851.0  2019.0   

REPORTYEAR         2017    2018    2019    2020    2021    2022  
Donor Type                                                       
Actual DBD       1610.0  1612.0  1557.0  1156.0  1243.0  1283.0  
Actual DCD        573.0  

#### Trasplants DBD vs Donants (pMp) - 2021

In [134]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el país 'Spain' y los años entre 2000 y 2022
df_spain = df[(df['COUNTRY'] == 'Spain') & (df['REPORTYEAR'].between(2000, 2022))]

# Agrupar por año y calcular los totales de DBD utilizados y DBD totales
totals_spain = df_spain.groupby('REPORTYEAR').agg({
    'Utilized DBD': 'sum',
    'Actual DBD': 'sum'
}).reset_index()

# Filtrar los años donde ambos valores (Utilized DBD y Actual DBD) son mayores que cero
totals_spain = totals_spain[(totals_spain['Utilized DBD'] > 0) | (totals_spain['Actual DBD'] > 0)]

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar los resultados en un archivo CSV
output_file = output_folder + 'totals_dbd_by_year_in_spain_filtered.csv'
totals_spain.to_csv(output_file, index=False)

# Mostrar los resultados por consola
print(totals_spain)


    REPORTYEAR  Utilized DBD  Actual DBD
3         2003           0.0      1387.0
4         2004           0.0      1424.0
5         2005           0.0      1475.0
7         2007           0.0      1462.0
8         2008           0.0      1500.0
9         2009        1317.0      1499.0
10        2010           0.0      1372.0
11        2011           0.0      1550.0
12        2012           0.0      1482.0
13        2013        1336.0      1496.0
14        2014        1297.0      1489.0
15        2015        1350.0      1537.0
16        2016        1334.0      1524.0
17        2017        1421.0      1610.0
18        2018        1398.0      1612.0
19        2019        1372.0      1557.0
20        2020        1035.0      1156.0
21        2021        1108.0      1243.0
22        2022        1139.0      1283.0


#### Trasplants DCD vs Donants (pMp) - 2021

In [135]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el país 'Spain' y los años entre 2000 y 2022
df_spain = df[(df['COUNTRY'] == 'Spain') & (df['REPORTYEAR'].between(2000, 2022))]

# Agrupar por año y calcular los totales de DCD utilizados y DCD totales
totals_spain = df_spain.groupby('REPORTYEAR').agg({
    'Utilized DCD': 'sum',
    'Actual DCD': 'sum'
}).reset_index()

# Filtrar los años donde ambos valores (Utilized DCD y Actual DCD) son mayores que cero
totals_spain = totals_spain[(totals_spain['Utilized DCD'] > 0) | (totals_spain['Actual DCD'] > 0)]

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar los resultados en un archivo CSV
output_file = output_folder + 'totals_dcd_by_year_in_spain_filtered.csv'
totals_spain.to_csv(output_file, index=False)

# Mostrar los resultados por consola
print(totals_spain)


    REPORTYEAR  Utilized DCD  Actual DCD
3         2003           0.0        56.0
4         2004           0.0        71.0
5         2005           0.0        71.0
6         2006           0.0        76.0
7         2007           0.0        88.0
8         2008           0.0        77.0
9         2009          83.0       107.0
10        2010           0.0       130.0
11        2011           0.0       117.0
12        2012           0.0       161.0
13        2013         119.0       159.0
14        2014         145.0       193.0
15        2015         255.0       314.0
16        2016         409.0       495.0
17        2017         485.0       573.0
18        2018         527.0       629.0
19        2019         616.0       745.0
20        2020         546.0       621.0
21        2021         582.0       662.0
22        2022         810.0       913.0


#### Total de trasplants vs Donants per País al 2021

In [136]:
import pandas as pd
import os

# Cargar el dataset
df = pd.read_csv('./data/GODT.csv')

# Filtrar datos solo para el país 'Spain' y todos los años disponibles
df_spain = df[(df['COUNTRY'] == 'Spain')]

# Agrupar por año y calcular los totales de DD utilizados y DD totales
totals_spain = df_spain.groupby('REPORTYEAR').agg({
    'Total Utilized DD': 'sum',
    'TOTAL Actual DD': 'sum'
}).reset_index()

# Filtrar los años donde ambos valores (Total Utilized DD y TOTAL Actual DD) son mayores que cero
totals_spain = totals_spain[(totals_spain['Total Utilized DD'] > 0) | (totals_spain['TOTAL Actual DD'] > 0)]

# Crear una carpeta de salida si no existe
output_folder = './data/out/spain/'
os.makedirs(output_folder, exist_ok=True)

# Guardar los resultados en un archivo CSV
output_file = output_folder + 'totals_dd_by_year_in_spain_filtered.csv'
totals_spain.to_csv(output_file, index=False)

# Mostrar los resultados por consola
print(totals_spain)


    REPORTYEAR  Total Utilized DD  TOTAL Actual DD
0         2000                0.0           1345.0
1         2001                0.0           1335.0
2         2002                0.0           1409.0
3         2003                0.0           1443.0
4         2004                0.0           1495.0
5         2005                0.0           1546.0
6         2006                0.0           1509.0
7         2007                0.0           1550.0
8         2008                0.0           1577.0
9         2009                0.0           1606.0
10        2010                0.0           1502.0
11        2011                0.0           1667.0
12        2012                0.0           1643.0
13        2013             1455.0           1655.0
14        2014             1442.0           1682.0
15        2015             1605.0           1851.0
16        2016             1743.0           2019.0
17        2017             1906.0           2183.0
18        2018             1925

## 5. Reflexió: Aprenentatge i limitacions

### Aprenentatges del conjunt de dades

- **Disparitat Geogràfica**: Disparitat mundial en l’accés als trasplants.
- **Variables Clau**: Hem identificat les variables clau que proporcionen informació detallada sobre l'activitat de donació i trasplantament d'òrgans, com ara:
    - Tipus específics de donants (DBD, DCD).
    - Trasplantaments d'òrgans (ronyó, fetge, cor, pulmó).
    - Bona taxa de trasplantaments que hi han a Europa i a Espanya i Catalunya en particular.

### M’hauria agradat

- Poder comparar diferents comunitats de l’Estat espanyol en quant a donants i trasplants.
- Veure la taxa de supervivència de cada un dels trasplants.

### Altres limitacions

- Infogram només puc fer servir 5 slides. El tamany és massa gran i el responsive no és gaire bó.no és gaire bó.



## 6. Diccionari de les variables

#### `REGION`  
Dada categòrica (que podem fer servir com a geogràfica). Identifica la regió mundial del registre. Per exemple tenim Africa, Europe, Eastern Mediterranean...

#### `COUNTRY`  
Dada categòrica (també pot fer-se servir com a geogràfica). És el país del registre. 

#### `REPORTYEAR`  
Dada que encara que sigui un nombre, la farem servir com a categòrica. Any del registre.

**Nota: A partir d'ara totes les dades són numèriques.**

#### `POPULATION`  
Població del país en milions d'habitants

#### `TOTAL Actual DD`  
Total de donants difunts.

#### `Actual DBD`  
Total de donants difunts per mort cerebral (Deceased Donor Brain Dead). 

#### `Actual DCD`  
Total de donants difunts per aturada cardíaca (Deceased Donor Cardiac Death).

#### `Total Utilized DD`  
Total de donants difunts utilitzats.

#### `Utilized DBD`
Donants difunts per mort cerebral usats.

#### `Utilized DCD`
Donants difunts per aturada cardíaca usats.

#### `DD Kidney Tx`
Trasplantaments de ronyó de donants difunts.

#### `LD Kidney Tx`
Trasplantaments de ronyó de donants vius.

#### `TOTAL Kidney Tx`
Total de trasplantaments de ronyó

#### `DD Liver Tx`
Trasplantaments de fetge de donants difunts.

#### `DOMINO Liver Tx` 
Trasplantaments de fetge tipus domino. Dos receptors per un sol fetge.

#### `LD Liver Tx`  
Trasplantaments de fetge de donants vius.

#### `TOTAL Liver TX`  
Total de trasplantaments de fetge.

#### `Total Heart TX`  
Total de trasplantaments de cor.

#### `DD Lung Tx`  
Trasplantaments de pulmó de donants difunts.

#### `LD Lung Tx`  
Trasplantaments de pulmó de donants vius.

#### `TOTAL Lung Tx` 
Total de trasplantaments de pulmó.

#### `Pancreas Tx`  
Trasplantaments de pàncrees.

#### `Kidney Pancreas Tx`  
Trasplantaments de ronyó i pàncrees.

#### `Small Bowel Tx`  
Trasplantaments d'intestí prim.

## Noves dades:


#### `Transplant Rate per 100k`  
Total de trasplantaments de cor.

#### `Transplant Rate per 100k`  
Taxa de trasplantaments d'òrgans per cada 100.000 habitants a cada país o regió.

#### `Donor Ratio by Transplant Type`  
Proporció d'utilització de donants (sigui DBD o DCD) per a cada tipus de trasplantament d'òrgan (ronyó, fetge, cor, etc.).

#### `Total Utilized LD`  
Total de donants difunts utilitzats, sumant LD Kidney Tx, LD Liver Tx i LD Lung Tx

Les dades seran enriquides, però no amb noves dades sinó calculant altres dades a partir de les que tenim.

## Cita de les dades

* Those 2023 data are based on the Global Observatory on Donation and Transplantation (GODT) data, produced by the WHO-ONT collaboration.
* Data of the WHO-ONT Global Observatory on Donation and Transplantation.
* [Use of the Data](https://www.transplant-observatory.org/uses-of-dataquoting-data/)

<!--Footer-->
 <div style="background: #333333;padding: 35px 0px;margin-top: 25px;">
    <div class="row">
     <div class="col-sm-12">
        <img src="http://materials.cv.uoc.edu/cdocent/common/img/logo-uoc-bottom.png" alt="Logo UOC" class="img-responsive" style="margin: 0 auto; display: block;">
    </div>
</div>
</div>
<!--/Footer-->